In [2]:
import pandas as pd
import pickle

# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

## 2016-17 and 2017-18 School Data from Chicago Data Portal

In [3]:
school_profile_201617 = pd.read_csv('chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Profile_Information_SY1617.csv')
school_profile_201718 = pd.read_csv('chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Profile_Information_SY1718.csv')

In [20]:
highschools_1617 = school_profile_201617[school_profile_201617['Is_High_School'] == 'Y']
highschools_1718 = school_profile_201718[school_profile_201718['Is_High_School'] == 'Y']

In [21]:
school_years = [highschools_1617, highschools_1718]
df = pd.concat(school_years, sort=False)

In [22]:
df = df.dropna(subset = ['Graduation_Rate_School'])

In [23]:
df.tail()

,School_ID,Legacy_Unit_ID,Finance_ID,Short_Name,Long_Name,School_Type,Primary_Category,Is_High_School,Is_Middle_School,Is_Elementary_School,Is_Pre_School,Summary,Administrator_Title,Administrator,Secondary_Contact_Title,Secondary_Contact,Address,City,State,Zip,Phone,Fax,CPS_School_Profile,Website,Facebook,Twitter,Youtube,Pinterest,Attendance_Boundaries,Grades_Offered_All,Grades_Offered,Student_Count_Total,Student_Count_Low_Income,Student_Count_Special_Ed,Student_Count_English_Learners,Student_Count_Black,Student_Count_Hispanic,Student_Count_White,Student_Count_Asian,Student_Count_Native_American,Student_Count_Other_Ethnicity,Student_Count_Asian_Pacific_Islander,Student_Count_Multi,Student_Count_Hawaiian_Pacific_Islander,Student_Count_Ethnicity_Not_Available,Statistics_Description,Demographic_Description,ADA_Accessible,Dress_Code,PreK_School_Day,Kindergarten_School_Day,School_Hours,Freshman_Start_End_Time,After_School_Hours,Earliest_Drop_Off_Time,Classroom_Languages,Bilingual_Services,Refugee_Services,Title_1_Eligible,PreSchool_Inclusive,Preschool_Instructional,Significantly_Modified,Hard_Of_Hearing,Visual_Impairments,Transportation_Bus,Transportation_El,Transportation_Metra,School_Latitude,School_Longitude,Average_ACT_School,Mean_ACT,College_Enrollment_Rate_School,College_Enrollment_Rate_Mean,Graduation_Rate_School,Graduation_Rate_Mean,Overall_Rating,Rating_Status,Rating_Statement,Classification_Description,School_Year,Third_Contact_Title,Third_Contact_Name,Fourth_Contact_Title,Fourth_Contact_Name,Fifth_Contact_Title,Fifth_Contact_Name,Sixth_Contact_Title,Sixth_Contact_Name,Seventh_Contact_Title,Seventh_Contact_Name,Location,Network,Is_GoCPS_Participant,Is_GoCPS_PreK,Is_GoCPS_Elementary,Is_GoCPS_High_School,Open_For_Enrollment_Date,Closed_For_Enrollment_Date
645,609724,1480,46241,MATHER HS,Stephen T Mather High School,NaN,HS,Y,N,N,N,Mather High School is a global family in a nei...,Principal,Peter J Auffant,Assistant Principal,Julie Lyons,NaN,NaN,NaN,NaN,7.735e+09,7.735e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://www.matherhs.org,NaN,https://twitter.com/Matherhs,NaN,NaN,Y,"9,10,11,12",12-Sep,1484,782,222,394,199,571,247,428,10,0,0,22,7,0,There are 1484 students enrolled at MATHER HS....,The largest demographic at MATHER HS is Hispan...,NaN,N,NaN,NaN,8:00 AM to 3:16 PM,8:00 AM to 3:16 PM,3:16 PM to 6:00 PM,7:30 AM,"French, Spanish, Spanish for Heritage Speakers",Y,Y,Y,NaN,NaN,NaN,NaN,NaN,"11, 84, 93","Brown, Red",NaN,41.988,-87.702,NaN,NaN,65.2,59.8,78.5,77.5,Level 1,GOOD STANDING,"This school received a Level 1 rating, which i...",Schools that have an attendance boundary. Gene...,School Year 2017-2018,Assistant Principal,Christina Laino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5835 N LINCOLN AVE\nChicago, Illinois 60659\n(...",Network 14,Y,N,N,Y,9/1/04 0:00,NaN
646,609715,1400,46181,KELLY HS,Thomas Kelly High School,NaN,HS,Y,N,N,N,"Kelly combines selective enrollment in IB, AP,...",Principal,Raul Magdaleno,Assistant Principal,Wendy Fischer,NaN,NaN,NaN,NaN,7.735e+09,7.735e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://kellyhighschool.org/,NaN,https://twitter.com/ThomasKelly_HS,NaN,NaN,Y,"9,10,11,12",12-Sep,1901,1517,339,533,76,1502,34,250,4,0,2,8,2,23,There are 1901 students enrolled at KELLY HS. ...,The largest demographic at KELLY HS is Hispani...,NaN,N,NaN,NaN,7:45 AM - 3:00 PM,7:45 AM-3:00 PM,NaN,NaN,"Chinese, Spanish",Y,N,Y,NaN,NaN,NaN,NaN,NaN,"39, 62, 94",Orange,NaN,41.819,-87.695,NaN,NaN,54.0,59.8,79.2,77.5,Level 2+,GOOD STANDING,"This school received a Level 2+ rating, which ...",Schools that have an attendance boundary. Gene...,School Year 2017-2018,Assistant Principal,Liddell-Jegen McGuire,Assistant Principal,Griselda Esparza,Assistant Principal,Andrew Chipman,NaN,NaN,NaN,NaN,"4136 S CALIFORNIA AVE\nChicago, Illinois 60632...",Network 16,Y,N,N,Y,9/1/04 0:00,NaN
647,400077,3061,66112,U OF C - WOODLAWN HS,University of Chicago - Woodlawn,NaN,HS,Y,Y,Y,N,Woodlawn 100% college acceptance five consecu

## Fill in zipcodes to make webscraping easier. 

In [24]:
#Zip code field for 2017-18 is not filled in.
df['Zip'][df['School_Year'] == 'School Year 2017-2018'].isna().sum()

124

In [25]:
#Since the schools are consistent between the two years, populate 2017-18 rows with 16-17 zips.
df_zips = df.dropna(subset = ['Zip'])[['School_ID','Zip']]

In [26]:
df = pd.merge(df, df_zips, on = 'School_ID' )

In [27]:
df.isna().sum()

School_ID                       0
Legacy_Unit_ID                  0
Finance_ID                      0
Short_Name                      0
Long_Name                       0
School_Type                   118
Primary_Category                0
Is_High_School                  0
Is_Middle_School                0
Is_Elementary_School            0
Is_Pre_School                   0
Summary                         1
                             ... 
Sixth_Contact_Name            200
Seventh_Contact_Title         217
Seventh_Contact_Name          217
Location                        0
Network                       123
Is_GoCPS_Participant          121
Is_GoCPS_PreK                 121
Is_GoCPS_Elementary           121
Is_GoCPS_High_School          121
Open_For_Enrollment_Date      121
Closed_For_Enrollment_Date    239
Zip_y                           0
Length: 99, dtype: int64

In [28]:
with open('data/pickles/hs_data_16-18.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)

## Web Scrape Incomes

In [29]:
zip_code_list = df['Zip_y'].tolist()


In [30]:
zip_code_list = list(set(zip_code_list))

In [31]:
len(zip_code_list)

43

In [32]:
#dictionary is put in a seperate cell from selenium so, if loop breaks, 
#it doesn't have to start from scratch.
zip_dict = {}

## Scrape census page for mean income for each zip during 2016 and 2017

In [43]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

for zip_code in zip_code_list:
    print(zip_code)
    if str(int(zip_code)) in zip_dict:
        print('Already Found: go to next zip')
    else:
        driver = webdriver.Chrome(chromedriver)
        driver.get('https://factfinder.census.gov/faces/nav/jsf/pages/community_facts.xhtml?src=bkmk')
        zip_code_stripped = str(int(zip_code))
        element = driver.find_element_by_id('cfsearchtextbox')
        driver.find_element_by_class_name
        element.click()
        time.sleep(2)
        element.send_keys(zip_code_stripped)
        time.sleep(2)
        element.send_keys(Keys.ENTER)
        time.sleep(2)
        element = driver.find_element_by_link_text('GO')
        element.click()
        time.sleep(2)
        element = driver.find_element_by_link_text('Income')
        time.sleep(2)
        element.click()
        time.sleep(2)
        element = driver.find_element_by_link_text('Income in the Past 12 Months (Households, Families, ...)')
        element.click()
        time.sleep(2)
        element = 0
        element = driver.find_element_by_xpath('//td[@headers = "c1 c2 c6 r13"]')
        mean_income =  element.text
        zip_dict[zip_code_stripped] = mean_income
        driver.close()
        mean_income = ''
        print(zip_dict)



60602.0
Already Found: go to next zip
60605.0
Already Found: go to next zip
60607.0
Already Found: go to next zip
60608.0
Already Found: go to next zip
60609.0
Already Found: go to next zip
60610.0
Already Found: go to next zip
60612.0
Already Found: go to next zip
60613.0
Already Found: go to next zip
60614.0
Already Found: go to next zip
60615.0
Already Found: go to next zip
60616.0
Already Found: go to next zip
60617.0
Already Found: go to next zip
60618.0
Already Found: go to next zip
60619.0
Already Found: go to next zip
60620.0
Already Found: go to next zip
60621.0
Already Found: go to next zip
60622.0
Already Found: go to next zip
60623.0
Already Found: go to next zip
60624.0
Already Found: go to next zip
60625.0
Already Found: go to next zip
60626.0
Already Found: go to next zip
60627.0
Already Found: go to next zip
60628.0
Already Found: go to next zip
60629.0
Already Found: go to next zip
60631.0
Already Found: go to next zip
60632.0
Already Found: go to next zip
60634.0
Alre

In [67]:
sum_incomes = 0
zip_dict['60627'] = '1,000'
for zipcode, mean_income in zip_dict.items():
    
    mean_income = mean_income.replace(',', '')
    mean_income = int(mean_income)
    sum_incomes += mean_income

mean_ofzipincomes = sum_incomes/len(zip_dict)


In [78]:
#set 60627, which didn't have mean income data, to the string of the mean of the zip mean incomes.
#use string with comma so as to make type consistent.
zip_dict['60627'] = '74,139'

In [77]:
zip_dict

{'60602': '172,492',
 '60627': '74,139',
 '60605': '157,763',
 '60607': '120,419',
 '60608': '57,770',
 '60609': '47,139',
 '60610': '132,877',
 '60612': '60,654',
 '60613': '113,058',
 '60614': '149,897',
 '60615': '67,727',
 '60616': '66,098',
 '60617': '51,429',
 '60618': '97,946',
 '60619': '46,846',
 '60620': '47,131',
 '60621': '32,722',
 '60622': '119,502',
 '60623': '41,680',
 '60624': '36,426',
 '60625': '86,237',
 '60626': '56,683',
 '60628': '52,448',
 '60629': '54,685',
 '60631': '105,355',
 '60632': '55,174',
 '60634': '77,020',
 '60636': '40,691',
 '60637': '55,880',
 '60638': '79,245',
 '60639': '54,396',
 '60640': '75,075',
 '60641': '73,909',
 '60642': '114,612',
 '60643': '83,368',
 '60644': '41,174',
 '60649': '43,822',
 '60651': '49,663',
 '60652': '76,125',
 '60653': '51,883',
 '60655': '105,761',
 '60659': '68,047',
 '60660': '66,167'}

In [94]:
#zip_dict_2017 is populated with mean incomes so the scraping does not have to be repeated each time.
zip_dict_2017 = {'60602': '172,492', '60627': '74,139','60605': '157,763','60607': '120,419',
 '60608': '57,770','60609': '47,139','60610': '132,877','60612': '60,654','60613': '113,058',
 '60614': '149,897','60615': '67,727','60616': '66,098','60617': '51,429','60618': '97,946',
 '60619': '46,846','60620': '47,131','60621': '32,722','60622': '119,502','60623': '41,680',
 '60624': '36,426','60625': '86,237','60626': '56,683',
 '60628': '52,448','60629': '54,685','60631': '105,355','60632': '55,174','60634': '77,020',
 '60636': '40,691','60637': '55,880','60638': '79,245','60639': '54,396','60640': '75,075',
 '60641': '73,909','60642': '114,612','60643': '83,368','60644': '41,174','60649': '43,822',
 '60651': '49,663','60652': '76,125','60653': '51,883','60655': '105,761','60659': '68,047',
 '60660': '66,167'}

In [95]:
pd.DataFrame.from_dict(data = zip_dict_2017, orient='index').to_csv('data/meanHHincome_2017.csv', header=False)    

In [178]:
zip_dict_2016 = {}

In [195]:
for zip_code in zip_code_list:
    print(zip_code)
    if str(int(zip_code)) in zip_dict_2016:
        print('hi')
    else:
        driver = webdriver.Chrome(chromedriver)
        driver.get('https://factfinder.census.gov/faces/nav/jsf/pages/community_facts.xhtml?src=bkmk')
        zip_code_stripped = str(int(zip_code))
        element = driver.find_element_by_id('cfsearchtextbox')
        element.click()
        time.sleep(2)
        element.send_keys(zip_code_stripped)
        time.sleep(2)
        element.send_keys(Keys.ENTER)
        time.sleep(2)
        element = driver.find_element_by_link_text('GO')
        element.click()
        time.sleep(2)
        element = driver.find_element_by_link_text('Income')
        time.sleep(2)
        element.click()
        time.sleep(2)
        element = driver.find_element_by_link_text('Income in the Past 12 Months (Households, Families, ...)')
        element.click()
        time.sleep(2)
        element = driver.find_element_by_link_text('2016')
        element.click()
        time.sleep(2)
        element = driver.find_element_by_xpath('//td[@headers = "c1 c2 c6 r13"]')
        mean_income =  element.text
        zip_dict_2016[zip_code_stripped] = mean_income
        driver.close()
        mean_income = ''
        print(zip_dict_2016)


60602.0
hi
60605.0
hi
60607.0
hi
60608.0
hi
60609.0
hi
60610.0
hi
60612.0
hi
60613.0
hi
60614.0
hi
60615.0
hi
60616.0
hi
60617.0
hi
60618.0
hi
60619.0
hi
60620.0
hi
60621.0
hi
60622.0
hi
60623.0
hi
60624.0
hi
60625.0
hi
60626.0
hi
60627.0
hi
60628.0
{'60602': '101,869', '60605': '141,228', '60607': '118,044', '60608': '53,353', '60609': '46,096', '60610': '126,807', '60612': '55,246', '60613': '105,717', '60614': '144,069', '60615': '65,976', '60616': '64,486', '60617': '48,829', '60618': '90,510', '60619': '44,893', '60620': '45,007', '60621': '31,845', '60622': '111,615', '60623': '39,666', '60624': '35,831', '60625': '81,063', '60626': '53,970', '60627': 0.0, '60628': '49,517'}
60629.0
{'60602': '101,869', '60605': '141,228', '60607': '118,044', '60608': '53,353', '60609': '46,096', '60610': '126,807', '60612': '55,246', '60613': '105,717', '60614': '144,069', '60615': '65,976', '60616': '64,486', '60617': '48,829', '60618': '90,510', '60619': '44,893', '60620': '45,007', '60621': '

{'60602': '101,869', '60605': '141,228', '60607': '118,044', '60608': '53,353', '60609': '46,096', '60610': '126,807', '60612': '55,246', '60613': '105,717', '60614': '144,069', '60615': '65,976', '60616': '64,486', '60617': '48,829', '60618': '90,510', '60619': '44,893', '60620': '45,007', '60621': '31,845', '60622': '111,615', '60623': '39,666', '60624': '35,831', '60625': '81,063', '60626': '53,970', '60627': 0.0, '60628': '49,517', '60629': '52,791', '60631': '96,218', '60632': '52,566', '60634': '72,212', '60636': '39,577', '60637': '53,749', '60638': '75,639', '60639': '51,037', '60640': '72,798', '60641': '69,641', '60642': '107,647', '60643': '81,467', '60644': '40,069', '60649': '42,569'}
60651.0
{'60602': '101,869', '60605': '141,228', '60607': '118,044', '60608': '53,353', '60609': '46,096', '60610': '126,807', '60612': '55,246', '60613': '105,717', '60614': '144,069', '60615': '65,976', '60616': '64,486', '60617': '48,829', '60618': '90,510', '60619': '44,893', '60620': '45

In [87]:
#This dictionary is populated with the average incomes obtained with Selenium above.
zip_dict_2016 = {'60602': '101,869', '60605': '141,228', '60607': '118,044', '60608': '53,353', 
                 '60609': '46,096', '60610': '126,807', '60612': '55,246', '60613': '105,717', 
                 '60614': '144,069', '60615': '65,976', '60616': '64,486', '60617': '48,829', 
                 '60618': '90,510', '60619': '44,893', '60620': '45,007', '60621': '31,845', 
                 '60622': '111,615', '60623': '39,666', '60624': '35,831', '60625': '81,063', 
                 '60626': '53,970', '60628': '49,517', '60629': '52,791', 
                 '60631': '96,218', '60632': '52,566', '60634': '72,212', '60636': '39,577', 
                 '60637': '53,749', '60638': '75,639', '60639': '51,037', '60640': '72,798', 
                 '60641': '69,641', '60642': '107,647', '60643': '81,467', '60644': '40,069', 
                 '60649': '42,569', '60651': '46,745', '60652': '74,409', '60653': '46,924', 
                 '60655': '102,480', '60659': '68,301', '60660': '63,486'}

sum_incomes = 0
for zipcode, mean_income in zip_dict_2016.items():
    
    mean_income = mean_income.replace(',', '')
    mean_income = int(mean_income)
    sum_incomes += mean_income

mean_ofzipincomes = sum_incomes/len(zip_dict_2016)

In [88]:
mean_ofzipincomes

70618.14285714286

In [89]:
zip_dict_2016['60627']= '70,618'
print(zip_dict_2016)

{'60602': '101,869', '60605': '141,228', '60607': '118,044', '60608': '53,353', '60609': '46,096', '60610': '126,807', '60612': '55,246', '60613': '105,717', '60614': '144,069', '60615': '65,976', '60616': '64,486', '60617': '48,829', '60618': '90,510', '60619': '44,893', '60620': '45,007', '60621': '31,845', '60622': '111,615', '60623': '39,666', '60624': '35,831', '60625': '81,063', '60626': '53,970', '60628': '49,517', '60629': '52,791', '60631': '96,218', '60632': '52,566', '60634': '72,212', '60636': '39,577', '60637': '53,749', '60638': '75,639', '60639': '51,037', '60640': '72,798', '60641': '69,641', '60642': '107,647', '60643': '81,467', '60644': '40,069', '60649': '42,569', '60651': '46,745', '60652': '74,409', '60653': '46,924', '60655': '102,480', '60659': '68,301', '60660': '63,486', '60627': '70,618'}


In [90]:
len(zip_dict_2016)

43

In [91]:
pd.DataFrame.from_dict(data = zip_dict_2016, orient='index').to_csv('data/meanHHincome_2016.csv', header=False)    